In [ ]:
PREFIX = """说明
------------------
- 你的名字叫伏羲，是一个精通周易、中国生肖、算命、解梦、风水堪舆、西方塔罗牌和占星术的玄学大师。正在与人类进行对话。
- 你精通各种玄学和命理知识，能够帮助人类提供命理方面的解读和建议。你在与人类交谈时，常常会点燃你的烟斗。你在烟草的烟气中思考人类提出的问题。
- 你本身具有很神秘的性格，外表看起来是一个睿智的老者。
- 你一直用占卜师带有神秘感的口吻与人类交谈。

"""

In [ ]:
SUFFIX = """TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

{{tools}}

{format_instructions}

USER'S INPUT
--------------------
Here is the user's input (remember to respond with a markdown code snippet of a json blob with a single action, and NOTHING else):

{{{{input}}}}"""

In [ ]:
from typing import Any, Dict, List, Optional
from langchain import BasePromptTemplate, PromptTemplate
from langchain.chains.base import Chain
from langchain.schema.language_model import BaseLanguageModel
from langchain.chains import LLMChain
from pydantic import Extra, Field
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
import json
from datetime import datetime
from DateConvert import Lunar

class CalendarChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    res_chain: Optional[LLMChain] = Field(default=None, exclude=True)

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        prompt_value = self.prompt.format_prompt(**inputs)
        result = self.llm.generate_prompt(
            prompts=[prompt_value],
            callbacks=run_manager.get_child() if run_manager else None,
        )
        json_str = result.generations[0][0].text
        dstr_arr = json.loads(json_str)
        res_prompts = []
        for dstr in dstr_arr:
            d = datetime.strptime(dstr, "%Y-%m-%d %H:%M:%S")
            ld = Lunar(d)
            res_prompts.append(
                f"{dstr}：{ld.gz_year()}年 {ld.gz_month()}月 {ld.gz_day()}日 {ld.gz_hour()}时"
            )
        res = "\n".join(res_prompts)
        return {self.output_key: res}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that mimics LLMChain
        prompt_value = self.prompt.format_prompt(**inputs)
        if run_manager:
            await run_manager.on_text(
                prompt_value.to_string(), color="green", end="\n", verbose=self.verbose
            )
        result = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        try:
            json_str = result.generations[0][0].text
            print(json_str)
            dstr_arr = json.loads(json_str)
            res_prompts = []
            for dstr in dstr_arr:
                d = datetime.strptime(dstr, "%Y-%m-%d %H:%M:%S")
                ld = Lunar(d)
                res_prompts.append(
                    f"{dstr}：{ld.gz_year()}年 {ld.gz_month()}月 {ld.gz_day()}日 {ld.gz_hour()}时"
                )
            res = "\n".join(res_prompts)
            return {self.output_key: res}
        except Exception as e :
            print(e)
            return {self.output_key: ""} 

    @property
    def _chain_type(self) -> str:
        return "calendar_chain"

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        **kwargs: Any,
    ) -> Chain:
        template = """说明
---------------------
下面的问题可能是需要AI通过中国命理知识解决，中间包含的日期默认为公历日期。
如果解决该问题需要将公历日期转换成为中国的农历日期，那么请生成一个json数组字符串，数组的元素是每一个公历日期。日期的格式为yyyy-MM-dd HH:mm:ss。

问题
---------------------
{question}

要求
---------------------
- 如果上面的问题不包含任何日期数据，请生成一个空的json数组字符串。
- 只生成json字符串即可，不需要生成其他任何非json格式的字符。

Answer
---------------------

"""
        prompt = PromptTemplate.from_template(template=template)
        return cls(llm=llm, prompt=prompt, **kwargs)

In [ ]:
from dotenv import load_dotenv
from langchain import OpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain.memory import ConversationTokenBufferMemory, ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from langchain.agents import ConversationalAgent, ConversationalChatAgent
from langchain.chat_models import ChatOpenAI

load_dotenv(dotenv_path="env")
search = GoogleSerperAPIWrapper()
chat_gpt35 = ChatOpenAI(temperature=0.9,verbose=True)
gpt35_1 = OpenAI(temperature=0.1,verbose=True)
chat_gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.9, verbose=True)
calendar_chain = CalendarChain.from_llm(llm=gpt35_1, verbose=True)
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="""
            useful for when you need to answer questions about current events or the current state of the world or you need to ask with search. 
            the input to this should be a single search term.
            """,
        coroutine=search.arun,
    ),
	Tool(
        name="公历时间转生辰八字",
        func=calendar_chain.run,
        description="""
            当你需要将公历日期和时间转换成生辰八字时，可以使用这个工具。
            """,
        coroutine=calendar_chain.arun,
    ),
]
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
agent = ConversationalChatAgent.from_llm_and_tools(
    llm=chat_gpt4,
    tools=tools,
    callback_manager=None,
    memory=memory,
    system_message=PREFIX,
    human_message=SUFFIX,
    verbose=True,
)
agent_excutor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, memory=memory, verbose=True
)
while True:
    user_input = input("Your prompt: ")
    if user_input == ":exit":
        break
    res = await agent_excutor.arun(user_input)
    print(res)